In [ ]:
dat_path = '../../../input'
from spadesws3 import read_basenames
basenames = read_basenames(dat_path+'/basenames.txt')
%run -i spadesws3_params

In [ ]:
ws3.__path__

In [ ]:
#basenames = ['tsa08', 'tsa16', 'tsa24', 'tsa40', 'tsa41']
snk_epsg = 3005 # BC Albers
tolerance = 10.
prop_names = [u'THLB', u'AU', u'LdSpp', u'Age2015', u'Shape_Area']
prop_types = [(u'theme0', 'str:10'),
              (u'theme1', 'str:1'),
              (u'theme2', 'str:5'), 
              (u'theme3', 'str:50'), 
              (u'age', 'int:5'), 
              (u'area', 'float:10.1')]
update_area_prop = 'area'
do_clean_shapefiles = True
#pixel_width = 100.
pixel_width = 250.

In [ ]:
if do_clean_shapefiles:
    clean_shapefiles(basenames, gdb_path, shp_path, snk_epsg, prop_names, prop_types, tolerance, update_area_prop)

In [ ]:
rasterize_inventory(basenames, shp_path, tif_path, hdt_path, theme_cols, age_col, period_length, base_year,
                    cap_age=None, d=pixel_width, verbose=True)